In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import keras.backend as K
# %pip install tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
df = pd.read_csv('../../Dataset/cleaned_used_cars_v3.csv')

In [3]:
df.shape

(236195, 25)

In [5]:
df.shape

(236195, 25)

In [6]:
df

,seller,offerType,price,abtest,yearOfRegistration,gearbox,powerPS,model,kilometer,brand,...,vehicleType.kleinwagen,vehicleType.kombi,vehicleType.limousine,vehicleType.suv,fuelType.benzin,fuelType.cng,fuelType.diesel,fuelType.elektro,fuelType.hybrid,fuelType.lpg
0,0,0,1500.0,0,2001.0,0,75.0,117,150000.0,37,...,1,0,0,0,1,0,0,0,0,0
1,0,0,3600.0,0,2008.0,0,69.0,102,90000.0,31,...,1,0,0,0,0,0,1,0,0,0
2,0,0,650.0,0,1995.0,0,102.0,11,150000.0,2,...,0,0,1,0,1,0,0,0,0,0
3,0,0,2200.0,0,2004.0,0,109.0,8,150000.0,25,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0.0,0,1980.0,0,50.0,40,40000.0,37,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236190,0,0,3999.0,0,2005.0,0,3.0,11,150000.0,2,...,0,1,0,0,0,0,1,0,0,0
236191,0,0,3200.0,1,2004.0,0,225.0,141,150000.0,30,...,0,0,1,0,1,0,0,0,0,0
236192,0,0,1150.0,1,2000.0,0,0.0,248,150000.0,24,...,0,0,0,0,1,0,0,0,0,0
236193,0,0,1199.0,0,2000.0,1,101.0,107,125000.0,32,...,0,0,0,0,1,0,0,0,0,0


In [7]:
def r2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [8]:
X = df.drop(['price'], axis=1)
y = df['price']
X

,seller,offerType,abtest,yearOfRegistration,gearbox,powerPS,model,kilometer,brand,notRepairedDamage,...,vehicleType.kleinwagen,vehicleType.kombi,vehicleType.limousine,vehicleType.suv,fuelType.benzin,fuelType.cng,fuelType.diesel,fuelType.elektro,fuelType.hybrid,fuelType.lpg
0,0,0,0,2001.0,0,75.0,117,150000.0,37,0,...,1,0,0,0,1,0,0,0,0,0
1,0,0,0,2008.0,0,69.0,102,90000.0,31,0,...,1,0,0,0,0,0,1,0,0,0
2,0,0,0,1995.0,0,102.0,11,150000.0,2,1,...,0,0,1,0,1,0,0,0,0,0
3,0,0,0,2004.0,0,109.0,8,150000.0,25,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,1980.0,0,50.0,40,40000.0,37,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236190,0,0,0,2005.0,0,3.0,11,150000.0,2,0,...,0,1,0,0,0,0,1,0,0,0
236191,0,0,1,2004.0,0,225.0,141,150000.0,30,1,...,0,0,1,0,1,0,0,0,0,0
236192,0,0,1,2000.0,0,0.0,248,150000.0,24,0,...,0,0,0,0,1,0,0,0,0,0
236193,0,0,0,2000.0,1,101.0,107,125000.0,32,0,...,0,0,0,0,1,0,0,0,0,0


In [11]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.1))
model.add(Dense(1))

print('--------------------Compile------------------------------------')
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse', metrics=['mae', r2])

# early_stop = EarlyStopping(monitor='val_loss', patience=100)

print('--------------------Fit------------------------------------')
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=100, epochs=1000)

print('--------------------Evaluate------------------------------------')
score = model.evaluate(X_test, y_test)
print(f'Deep Learning Regression Mean Absolute Error: {score[1]:.2f}')
print(f'Deep Learning Regression R-squared Score: {score[2]:.2f}')

--------------------Compile------------------------------------
--------------------Fit------------------------------------
Epoch 1/1000
1512/1512 [==============================] - 7s 3ms/step - loss: 19297570.0000 - mae: 3429.8333 - r2: -0.2150 - val_loss: 15949222.0000 - val_mae: 3145.7539 - val_r2: -0.0162
Epoch 2/1000
1512/1512 [==============================] - 4s 3ms/step - loss: 13982062.0000 - mae: 2942.1443 - r2: 0.1191 - val_loss: 12573136.0000 - val_mae: 2793.6577 - val_r2: 0.1954
Epoch 3/1000
1512/1512 [==============================] - 4s 3ms/step - loss: 12904153.0000 - mae: 2817.9548 - r2: 0.1860 - val_loss: 12497183.0000 - val_mae: 2895.8979 - val_r2: 0.1971
Epoch 4/1000
1512/1512 [==============================] - 4s 3ms/step - loss: 12681611.0000 - mae: 2782.7405 - r2: 0.1973 - val_loss: 12183748.0000 - val_mae: 2825.9597 - val_r2: 0.2178
Epoch 5/1000
1512/1512 [==============================] - 4s 3ms/step - loss: 12452816.0000 - mae: 2753.5613 - r2: 0.2137 - val_lo

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1600      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 4,225
Trainable params: 4,225
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.get_weights()

[array([[ 1.2477881e-01,  2.4134234e-01,  2.1731248e-01, ...,
         -2.0761526e-01, -1.3671795e-01,  2.1369651e-01],
        [-8.7232888e-03, -3.1096005e+00,  4.1024942e+00, ...,
          6.6599309e-02,  5.9360111e-01,  7.5994641e-02],
        [-3.9493710e-02, -9.0432793e-01, -1.3427007e-02, ...,
          1.6678426e-01,  3.4596634e-01, -2.1473847e-01],
        ...,
        [-2.0161083e-01, -8.3537846e+00,  1.8468847e+00, ...,
         -6.4237162e-02,  8.8470640e+00,  9.2618763e-03],
        [ 2.0347723e-01,  6.2398696e+00, -1.1306380e+01, ...,
         -2.3413770e-01,  1.3756638e+01, -1.9989401e-02],
        [-1.2101561e-02, -1.0543760e+01,  1.1891617e+01, ...,
          3.9495260e-02, -2.1964743e+01, -4.0726915e-02]], dtype=float32),
 array([ 0.        ,  0.7641127 , -0.00834176, -1.0313848 , -0.0024261 ,
        -0.25292933, -0.02608551,  0.20732503,  0.2899738 , -0.9104436 ,
        -0.68517077, -1.1559947 , -0.98775846,  0.        ,  0.75346196,
        -0.51725113,  0.3949246

In [14]:
predictions = model.predict(X_test)

# Print the predicted prices
print(predictions)

1477/1477 [==============================] - 2s 2ms/step
[[11084.997 ]
 [ 1827.2302]
 [ 6120.347 ]
 ...
 [ 8243.836 ]
 [ 1464.9064]
 [ 3150.7566]]


In [15]:
y_test

57150     11800.0
174552     1100.0
232771     4360.0
3924       6999.0
3195       1899.0
           ...   
210308      555.0
20689      7750.0
94695      2100.0
159766     2499.0
229751     7200.0
Name: price, Length: 47239, dtype: float64

In [16]:
import plotly.express as px
import pandas as pd

# create a dataframe with the training history
df_graph = pd.DataFrame(history.history)

# create line charts for loss and MAE
fig = px.line(df_graph, x=df_graph.index+1, y=['loss', 'val_loss'], labels={'x':'Epoch', 'value':'Loss'},
              title='Training and Validation Loss')
fig.update_traces(mode='lines')
fig.show()

fig = px.line(df_graph, x=df_graph.index+1, y=['mae', 'val_mae'], labels={'x':'Epoch', 'value':'MAE'},
              title='Training and Validation MAE')
fig.update_traces(mode='lines')
fig.show()